# Desenvolvimento do modelo de predição

In [26]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature

Em MacOS é necessário instalar o liomp para a biblioteca XGBoost funcionar.

## Obtendo dados do dataset

In [2]:
ds = datasources.get('michelpf/fiap-ds-mlops-laptop-pricing-brl', 'processed')

Accessing as michelpf

In [3]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,86240413,https://dagshub.com/api/v1/repos/michelpf/fiap...,text/plain,79387


In [4]:
res = ds.head()
for dp in res:
    print(dp.path_in_repo)
    print(dp.download_url)
    dataset_url = dp.download_url

Output()

data/processed/laptop-price-brl-processed.csv
https://dagshub.com/api/v1/repos/michelpf/fiap-ds-mlops-laptop-pricing-brl/raw/main/data/processed/laptop-price-brl-processed.csv


In [22]:
df = pd.read_csv(dataset_url)

In [23]:
df.head()

,price,brand_asus,brand_dell,brand_hp,brand_lenovo,brand_other,processor_brand_amd,processor_brand_intel,processor_brand_m1,processor_name_core i3,...,ssd_2048,ssd_3072,graphic_card_gb_4,graphic_card_gb_8,graphic_card_gb_16,graphic_card_gb_32,ram_type_ddr4,ram_type_other,os_bit_32-bit,os_bit_64-bit
0,2321,1,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
1,2613,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
2,2680,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
3,4689,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
4,1808,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1


## Desenvolvimento de modelos

Rastreamento de experimentos utilizando MLflow.

In [8]:
dagshub.init(repo_owner='michelpf', repo_name='fiap-ds-mlops-laptop-pricing-brl', mlflow=True)

Initialized MLflow to track repo "michelpf/fiap-ds-mlops-laptop-pricing-brl"

Repository michelpf/fiap-ds-mlops-laptop-pricing-brl initialized!

In [9]:
mlflow.autolog()

2025/05/11 22:21:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/05/11 22:21:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/11 22:21:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [11]:
features = list(df.columns)

features.remove("price")

features

['brand_asus',
 'brand_dell',
 'brand_hp',
 'brand_lenovo',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_core i7',
 'processor_name_other',
 'processor_name_ryzen 5',
 'processor_name_ryzen 7',
 'os_other',
 'os_windows',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight',
 'warranty_0',
 'warranty_1',
 'warranty_2',
 'warranty_3',
 'touchscreen_0',
 'touchscreen_1',
 'ram_gb_4',
 'ram_gb_8',
 'ram_gb_16',
 'ram_gb_32',
 'hdd_0',
 'hdd_512',
 'hdd_1024',
 'hdd_2048',
 'ssd_0',
 'ssd_128',
 'ssd_256',
 'ssd_512',
 'ssd_1024',
 'ssd_2048',
 'ssd_3072',
 'graphic_card_gb_4',
 'graphic_card_gb_8',
 'graphic_card_gb_16',
 'graphic_card_gb_32',
 'ram_type_ddr4',
 'ram_type_other',
 'os_bit_32-bit',
 'os_bit_64-bit']

In [12]:
X = df[features]
X

,brand_asus,brand_dell,brand_hp,brand_lenovo,brand_other,processor_brand_amd,processor_brand_intel,processor_brand_m1,processor_name_core i3,processor_name_core i5,...,ssd_2048,ssd_3072,graphic_card_gb_4,graphic_card_gb_8,graphic_card_gb_16,graphic_card_gb_32,ram_type_ddr4,ram_type_other,os_bit_32-bit,os_bit_64-bit
0,1,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,1
1,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,1
2,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,1
3,1,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
775,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
776,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
777,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1


In [13]:
y = df["price"]
y

0       2321
1       2613
2       2680
3       4689
4       1808
       ...  
774     9111
775     9714
776    10049
777     9580
778     3852
Name: price, Length: 779, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=42)

In [30]:
def evaluate_and_log_model(model_name, model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    # Inferir a assinatura automaticamente
    signature = infer_signature(X_test, predictions)

    mlflow.sklearn.log_model(model, "model", signature=signature, input_example=X_test[:5])

    print(f"Resultados para {model_name}:")
    print(f" MSE: {mse:.4f}")
    print(f" MAE: {mae:.4f}")
    print(f" R2: {r2:.4f}")
    print(f" MAPE: {mape:.4%}")

In [31]:
with mlflow.start_run(run_name="Ridge_Regression"):
    param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
    ridge = Ridge()
    grid_search = GridSearchCV(ridge, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", best_model.alpha)
    evaluate_and_log_model("Ridge Regression", best_model, X_test, y_test)

2025/05/11 22:48:32 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


Resultados para Ridge Regression:
 MSE: 2995948.9648
 MAE: 1105.3962
 R2: 0.6458
 MAPE: 21.4528%
🏃 View run Ridge_Regression at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/7984e276c15047fe827ad1149b6043c9
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0


In [32]:
with mlflow.start_run(run_name="DecisionTree_Regressor"):
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10]
    }
    tree = DecisionTreeRegressor(random_state=42)
    grid_search = GridSearchCV(tree, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_min_samples_split", best_model.min_samples_split)
    evaluate_and_log_model("Decision Tree Regressor", best_model, X_test, y_test)

2025/05/11 22:49:51 INFO mlflow.sklearn.utils: Logging the 5 best runs, 7 runs will be omitted.


Resultados para Decision Tree Regressor:
 MSE: 4097876.5800
 MAE: 1097.0816
 R2: 0.5155
 MAPE: 17.2544%
🏃 View run DecisionTree_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/5ff8ee205902405b9bfb9ee59c121332
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0


In [33]:
from sklearn.neural_network import MLPRegressor

with mlflow.start_run(run_name="MLP_Regressor"):
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50)],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001, 0.001, 0.01]
    }
    mlp = MLPRegressor(max_iter=500, random_state=42)
    grid_search = GridSearchCV(mlp, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_hidden_layer_sizes", best_model.hidden_layer_sizes)
    mlflow.log_param("best_activation", best_model.activation)
    mlflow.log_param("best_alpha", best_model.alpha)
    evaluate_and_log_model("MLP Regressor", best_model, X_test, y_test)

/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/neural_networ

Resultados para MLP Regressor:
 MSE: 3499286.6185
 MAE: 1221.2536
 R2: 0.5863
 MAPE: 24.9985%
🏃 View run MLP_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/0984162ce86d4b72b01403cf670990f3
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0


In [34]:
with mlflow.start_run(run_name="XGBoost_Regressor"):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.2, 0.3]
    }
    xgb = XGBRegressor(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("XGBoost Regressor", best_model, X_test, y_test)

2025/05/11 22:52:48 INFO mlflow.sklearn.utils: Logging the 5 best runs, 43 runs will be omitted.


Resultados para XGBoost Regressor:
 MSE: 3131315.5000
 MAE: 999.0272
 R2: 0.6298
 MAPE: 16.2887%
🏃 View run XGBoost_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/de3558cf4aef4e5885cfed6f01d2f778
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0


In [35]:
with mlflow.start_run(run_name="LightGBM_Regressor"):
    param_grid = {
        'num_leaves': [50, 100],
        'learning_rate': [0.05, 0.1],
        'n_estimators': [100, 200]
    }
    model = lgb.LGBMRegressor(random_state=42, verbosity=-1)
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_absolute_percentage_error', cv=5)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Log
    mlflow.log_params(grid_search.best_params_)
    evaluate_and_log_model("LightGBM Regressor", best_model, X_test, y_test)

/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-laptop-pricing-brl/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be r

Resultados para LightGBM Regressor:
 MSE: 3172488.2757
 MAE: 1034.5018
 R2: 0.6249
 MAPE: 17.9483%
🏃 View run LightGBM_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/b480135d0a1a49e3a56e25a6286585ed
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0


In [36]:
with mlflow.start_run(run_name="Support_Vector_Regressor"):
    param_grid = {
        'C': [0.1, 1.0, 10.0],
        'epsilon': [0.01, 0.1, 0.2],
        'kernel': ['linear', 'rbf']
    }
    model = SVR()
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_absolute_percentage_error', cv=5)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_params(grid_search.best_params_)
    evaluate_and_log_model("SVR", best_model, X_test, y_test)

2025/05/11 22:54:32 INFO mlflow.sklearn.utils: Logging the 5 best runs, 13 runs will be omitted.


Resultados para SVR:
 MSE: 5487340.6490
 MAE: 1323.3072
 R2: 0.3512
 MAPE: 22.4644%
🏃 View run Support_Vector_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0/runs/d0e14d4eadd149de8135430444d4e8a2
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-laptop-pricing-brl.mlflow/#/experiments/0
